In [3]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


In [4]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Configure Selenium options
chrome_opts = Options()
chrome_opts.add_argument("--headless")
chrome_opts.add_argument("--no-sandbox")
chrome_opts.add_argument("--disable-dev-shm-usage")

# Spoof user-agent to avoid 403 Forbidden
chrome_opts.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/115.0.0.0 Safari/537.36"
)

# Launch browser
browser = webdriver.Chrome(options=chrome_opts)

# Open IMDb Top 250
browser.get("https://www.imdb.com/chart/top/")
time.sleep(5)  # wait for page to load

film_list = []
movie_cards = browser.find_elements(By.CSS_SELECTOR, ".ipc-metadata-list-summary-item")

# Extract movie details
for rank, card in enumerate(movie_cards, start=1):
    try:
        name = card.find_element(By.CSS_SELECTOR, "h3").text
        release_year = card.find_element(By.CSS_SELECTOR, ".cli-title-metadata-item").text
        score = card.find_element(By.CSS_SELECTOR, ".ipc-rating-star--imdb").text.split()[0]
        film_list.append([rank, name, release_year, score])
    except Exception as e:
        print(f"Skipping a card due to error: {e}")

browser.quit()

# Save as DataFrame
imdb_table = pd.DataFrame(film_list, columns=["Position", "Movie", "Release Year", "Rating"])
imdb_table.to_csv("imdb_top250.csv", index=False)
print(imdb_table.head())

   Position                        Movie Release Year Rating
0         1  1. The Shawshank Redemption         1994    9.3
1         2             2. The Godfather         1972    9.2
2         3           3. The Dark Knight         2008    9.1
3         4     4. The Godfather Part II         1974    9.0
4         5              5. 12 Angry Men         1957    9.0
